# So here is the plan
1. Use a known (first appearing) department name combined with it's - also known - page number to extract font metadata `FM` for department headings in the file
2. Use `FM` to find the rest of instances of a department name and use them to divide sections of the catalog. 
3. Withing each section, to find every course belonging to that section, run regex using only one variable `course ID` and using following strategy: 
```
find course ID, match everything (non greedy) until next course ID
```


In [198]:
# files & setup
# !pip install pdfminer pypdf2 tqdm openpyxl pandas gspread df2gspread psycopg2
# !exa -l data
# !exa data -T
# [f'{d},{ff}'for d,_,f in os.walk('data') for ff in f]

In [5]:
import io
import re
import os
import sys
from collections import defaultdict

import pandas as pd
from tqdm.notebook import tqdm

from utils.dataframe import gdrive_sheets, df2gsheet, gsheet, db2df, get_mapping
from utils.pdf_processing import (get_all_data, 
                                  get_text_object_details, 
                                  pdfminer_to_text, pypdf2txt)
from utils.text_processing import find, get_text_from_position

In [33]:
# done = False
# for directory,_,filenames in os.walk('data'):
#     if directory.lstrip('data/') in parse_only:
#         for filename in filenames:
#             data = pdf_to_text(directory+'/'+filename)
#             done = True
#             break
#     if done: break

# Arkansas

In [2]:
df_arkansas = db2df(college='Arkansas')

In [4]:
df_arkansas.head()

,db_id,filepath,filename,college,pdfminer,pdfminer_detailed,pypdf2
0,68,data/Arkansas/Arkansas_2017-2018.pdf,Arkansas_2017-2018.pdf,Arkansas,Master of Business Administration (MBAD) (p. 7...,"[{'page': 1, 'tboxes': [{'text': 'Master of Bu...",538\n \n \nAccounting (ACCT)\nLatin (LA...
1,69,data/Arkansas/Arkansas_2002-2003.pdf,Arkansas_2002-2003.pdf,Arkansas,257\n\nE\nS\nR\nU\nO\nC\n\nS\nN\nO\nI\nT\nP\nI...,"[{'page': 1, 'tboxes': [{'text': '257\n', 'fon...",\n\n\n\n\n˛\n˛\n#$\n\n#$\n#'\n#'\n\n#\n\n#\n\n...
2,70,data/Arkansas/Arkansas_2012-2013.pdf,Arkansas_2012-2013.pdf,Arkansas,Course Descriptions\n\nHow to Read a Course De...,"[{'page': 1, 'tboxes': [{'text': 'Course Descr...","University of Arkansas, Fayetteville \n307How ..."
3,71,data/Arkansas/Arkansas_2010-2011.pdf,Arkansas_2010-2011.pdf,Arkansas,Course Descriptions\n\nHow to Read a Course De...,"[{'page': 1, 'tboxes': [{'text': 'Course Descr...","University of Arkansas, Fayetteville \n306How ..."
4,72,data/Arkansas/Arkansas_2011-2012.pdf,Arkansas_2011-2012.pdf,Arkansas,Course Descriptions\n\nHow to Read a Course De...,"[{'page': 1, 'tboxes': [{'text': 'Course Descr...","University of Arkansas, Fayetteville \n303How ..."


In [35]:
pd.concat(df_arkansas.pdfminer.map(lambda s: re.findall(r"(\n\n?[A-Z]{3,4}\s[0-9]{3,4}.*)", s)).map(pd.DataFrame).values[0:2])

,0
0,\nADLL 5113. Perspectives in Adult Education (...
1,\n\nADLL 5123. Principles and Practices of Adu...
2,\n\nADLL 5133. Curriculum Development in ABE a...
3,\n\nADLL 5143. Instructional Strategies and As...
4,\n\nADLL 5153. Organization and Administration...
...,...
469,\nENGL 3253)
470,\nENGL 3273)
471,\nZOOL 1613. UNIVERSITY CORE COURSE
472,\nZOOL 2443 and ZOOL 2441L may not be counted ...


# Bowdowin

In [110]:
# pdf2txt doesnt work
bowdowin_sample = pdf_to_text("data/Bowdoin/Bowdoin_2019-2020.pdf")
bs = bowdowin_sample

In [346]:
# find = "AFRS 1012"
# bs[bs.index(find)-10:bs.index(find)+2000]
bowdoin_dfs = []
for file in tqdm(os.listdir('data/Bowdoin')):
    text = pdf_to_text(f"data/Bowdoin/{file}")
    if '202' in file:
        data = [{'id':r[1],'title':r[3],'sub':r[4],'desc':r[6],'num':r[5],'catalog':file} 
            for r in re.findall(
                r"("
                    r"\n\n([A-Z]{4}(\s|\\xa0)[0-9]{4})"  # course id
                    r"(.*?)\n"                           # course title
                    r"(.*?)"                             # course subtitle
                    r"\s([0-9]{1,3})\."                  # course enrolment limit
                    r"((?:(?![\n]{1}[A-Z]{4}).)*)"       # course description 
                                                          # UNTIL next course id 
                                                          # but not including it
                r")",
                text,
                re.DOTALL)
        ]
    else: #pre 2020
        data = [{'id':r[1],'title':r[3],'sub':None,'desc':r[4],'num':None,'catalog':file} 
                for r in re.findall(
                r"("
                    r"\n([0-9]{2,3}[a-z]{0,1})(\.\s|\s-)"  # course id
                    r"(.*?)\n"                           # course title
                    r"((?:(?!\n([0-9]{2,3}[a-z]{0,1})(\.\s|\s-)).)*)"  # UNTIL next course id but not including
                r")",
                text,
                re.DOTALL)
        ]
        
    bowdoin_dfs.append(pd.DataFrame(data))

In [365]:
df_bowdoin

,id,title,sub,desc,num,catalog
0,10b,Racism. Fall 2010. Spring 2011. Roy Partridge....,None,12c. Affirmative Action and United States Soci...,None,Bowdoin_2010-2011.pdf
1,13c,From Montezuma to Bin Laden: Globalization and...,None,Gordon. (Same as History 16.),None,Bowdoin_2010-2011.pdf
2,16c,Fictions of Freedom. Fall 2010. Tess Chakkalak...,None,[25c. The Civil War in Film. (Same as History ...,None,Bowdoin_2010-2011.pdf
3,101c,Introduction to Africana Studies. Every fall. ...,None,Focuses on major humanities and social science...,None,Bowdoin_2010-2011.pdf
4,107c,ESD. Introduction to the Black Novel in the U...,None,Chakkalakal.Introduces students to the literar...,None,Bowdoin_2010-2011.pdf
...,...,...,...,...,...,...
840,320c,Theater Styles. Every third year. Fall 2009. D...,None,An advanced acting class that explores issues ...,None,Bowdoin_2009-2010.pdf
841,321c,Comedy in Performance. Every third year. Fall ...,None,"Looks at several facets of comedy on stage, fr...",None,Bowdoin_2009-2010.pdf
842,322c,Ensemble: Theater and Dance Collaborative...,None,DePaRtMent.Experienced theater and dance stude...,None,Bowdoin_2009-2010.pdf
843,323c,Acting Shakespeare. Spring 2010. abiGail killeen.,None,An advanced-level acting course dedicated to t...,None,Bowdoin_2009-2010.pdf


In [368]:
df_bowdoin = pd.concat(bowdoin_dfs)
df_bowdoin['title'] = df_bowdoin['title'].str.replace('\n','')
# df_bowdoin['title'] = df_bowdoin['title'].str.replace(r'\x','')
df_bowdoin['desc'] = df_bowdoin['desc'].str.replace('\n','')
# df_bowdoin['desc'] = df_bowdoin['desc'].str.replace(r'\x','')
df_bowdoin = df_bowdoin.applymap(lambda x: x.encode('unicode_escape').
                 decode('utf-8') if isinstance(x, str) else x)
with pd.ExcelWriter('data/bowdoin.xlsx') as writer: 
    df_bowdoin.to_excel(writer, sheet_name='data')
df_bowdoin

,id,title,sub,desc,num,catalog
0,10b,Racism. Fall 2010. Spring 2011. Roy Partridge....,None,12c. Affirmative Action and United States Soci...,None,Bowdoin_2010-2011.pdf
1,13c,From Montezuma to Bin Laden: Globalization and...,None,Gordon. (Same as History 16.),None,Bowdoin_2010-2011.pdf
2,16c,Fictions of Freedom. Fall 2010. Tess Chakkalak...,None,[25c. The Civil War in Film. (Same as History ...,None,Bowdoin_2010-2011.pdf
3,101c,Introduction to Africana Studies. Every fall. ...,None,Focuses on major humanities and social science...,None,Bowdoin_2010-2011.pdf
4,107c,ESD. Introduction to the Black Novel in the U...,None,Chakkalakal.Introduces students to the literar...,None,Bowdoin_2010-2011.pdf
...,...,...,...,...,...,...
840,320c,Theater Styles. Every third year. Fall 2009. D...,None,An advanced acting class that explores issues ...,None,Bowdoin_2009-2010.pdf
841,321c,Comedy in Performance. Every third year. Fall ...,None,"Looks at several facets of comedy on stage, fr...",None,Bowdoin_2009-2010.pdf
842,322c,Ensemble: Theater and Dance Collaborative...,None,DePaRtMent.Experienced theater and dance stude...,None,Bowdoin_2009-2010.pdf
843,323c,Acting Shakespeare. Spring 2010. abiGail killeen.,None,An advanced-level acting course dedicated to t...,None,Bowdoin_2009-2010.pdf


# Ohio State

In [49]:
df_ohiostate = db2df(college='Ohio State')

In [50]:
df_ohiostate.head()

,db_id,filepath,filename,college,pdfminer,pdfminer_detailed,pypdf2
0,94,data/Ohio State/Ohio State_2016-2017.pdf,Ohio State_2016-2017.pdf,Ohio State,Course Offerings Bulletin \n\n2016—2017 \n\n ...,"[{'page': 1, 'tboxes': [{'text': 'Course Offer...",\n \n\n\n\n \ni 2016 Œ 2017 Course Offerings ...
1,95,data/Ohio State/Ohio State_2014-2015.pdf,Ohio State_2014-2015.pdf,Ohio State,Course Offerings Bulletin\n2014–2015\n\n \n\n...,"[{'page': 1, 'tboxes': [{'text': 'Course Offer...",Course O˜erings Bulletin2014Œ2015i \n 2014\n ...
2,96,data/Ohio State/Ohio State_2009-2010.pdf,Ohio State_2009-2010.pdf,Ohio State,Accounting and Management Information Syste...,"[{'page': 1, 'tboxes': []}, {'page': 2, 'tboxe...","3Academic Affairs\n203 Bricker Hall, 190 North..."
3,97,data/Ohio State/Ohio State_2020-2021.pdf,Ohio State_2020-2021.pdf,Ohio State,Course Offerings Bulletin\n2020-2021\n\n \n\n...,"[{'page': 1, 'tboxes': [{'text': 'Course Offer...",Course Offerings Bulletin\n2020\n-\n2021\ni 2...
4,98,data/Ohio State/Ohio State_2006-2007.pdf,Ohio State_2006-2007.pdf,Ohio State,\n\n \n\nAccounting and Management ...,"[{'page': 1, 'tboxes': [{'text': ' \...","15Academic Affairs\n203 Bricker Hall, 190 Nort..."


In [1]:
from utils.pdf_processing import get_all_data 

In [9]:
ohio_sample = pypdf2txt("data/Ohio State/course_catalog_2010_2011.pdf")

In [2]:
d=get_all_data("data/Ohio State/Ohio State_2014-2015.pdf", 4)

In [3]:
from itertools import chain
c = lambda z: list(chain.from_iterable(z))

In [8]:
import sys
sys.getsizeof(d['pdfminer'])
# ['pdfminer']
d.keys()

dict_keys(['filepath', 'filename', 'college', 'pdfminer', 'pdfminer_detailed', 'fonts', 'pypdf2', 'pypdf2_detailed'])

In [2]:
# ''.join(c([p['tboxes'] for p in d['pdfminer_detailed']]))

In [7]:
print(''.join((list(chain.from_iterable([l[1] for l in d["pdfminer_detailed"]])))))

KeyError: 1

In [3]:
# print(list(chain.from_iterable([l[0] for l in d["pdfminer_detailed"]])))

In [5]:
print(list(chain.from_iterable(d["pdfminer_detailed"][0])))
print('\n\n\n\n\n\n\n\n')


[24.0, 24.0, 24.0, 24.0, 24.0, 24.0, 24.0, 24.0, 24.0, 24.0, 24.0, 24.0, 24.0, 24.0, 24.0, 24.0, 24.0, 24.0, 24.0, 24.0, 24.0, 24.0, 24.0, 24.0, 24.0, 24.0, 24.0, 24.0, 24.0, 24.0, 24.0, 24.0, 24.0, 'C', 'o', 'u', 'r', 's', 'e', ' ', 'O', 'ff', 'e', 'r', 'i', 'n', 'g', 's', ' ', 'B', 'u', 'l', 'l', 'e', 't', 'i', 'n', '2', '0', '1', '4', '–', '2', '0', '1', '5']











In [ ]:
print(list(chain.from_iterable(d["pdfminer_detailed"][1])))
print('\n\n\n\n\n\n\n\n')
print(d["pdfminer"])

In [7]:
print(len(d["pdfminer_detailed"]))
print(len(d["pdfminer"]))

10
4400049


In [154]:
ohio_dfs = []
for file in tqdm(os.listdir('data/Ohio State')):
    # better to use pdf2txt, otherwise the number to the left of letter U
    # sometimes flotes away from the course it is assigned to
    ohs2 = pdf2txt(f"data/Ohio State/{file}")

    data = [{'id':r[1],'title':r[6],'desc':r[7],'num':r[8],'catalog':file} 
     for r 
     in re.findall(
         r"("
            r"\n("                           # course ID
                r"([0-9]{4}\.[0-9A-Z]{3})"     # e.g. 2367.07S
                r"|([0-9]{4}\.[0-9]{2})"       # e.g. 2367.04
                r"|([0-9]{4}[H])"              # e.g. 4780H
                r"|([0-9]{4})"                 # e.g. 3080
            r")"
           r"\n([A-Za-z\s]*)"                # course title
           r"\nU\n"                          # letter U
           r"(.*?)"                          # course desrciption
           r"\n([0-9]{1,2})"                 # number to the right of letter U
         r")",
         ohs2, 
         re.DOTALL) 
    ]
    ohio_dfs.append(pd.DataFrame(data))    

In [155]:
df_ohio = pd.concat(ohio_dfs)
df_ohio['title'] = df_ohio['title'].str.replace('\n','')
df_ohio['desc'] = df_ohio['desc'].str.replace('\n','')
with pd.ExcelWriter('data/ohio.xlsx') as writer: 
    df_ohio.to_excel(writer, sheet_name='data')
df_ohio

,id,title,desc,num,catalog
0,2000,Foundations of Accounting,A survey of accounting for non-business studen...,3,course_catalog_2015_2016.pdf
1,2200,Introduction to Accounting I,Preparation and use of accounting reports for ...,3,course_catalog_2015_2016.pdf
2,2200H,Introduction to Accounting I,Structure and use of financial reports; role o...,3,course_catalog_2015_2016.pdf
3,2300,Introduction to Accounting II,The uses of accounting reports by managers to ...,20,course_catalog_2015_2016.pdf
4,2300H,Introduction to Accounting II,Use of accounting information in management co...,3,course_catalog_2015_2016.pdf
...,...,...,...,...,...
3899,4721,Studies in Yiddish Literature,"Advanced study of specific literary periods, f...",3,Course Bulletin 2019-2020.pdf
3900,4998,Undergraduate Research,Investigation of problems in various fields of...,1,Course Bulletin 2019-2020.pdf
3901,4998H,Honors Research,Investigation of problems in various fields of...,3,Course Bulletin 2019-2020.pdf
3902,4999,Undergraduate Research Thesis,Research for undergraduate thesis.Prereq: Perm...,1,Course Bulletin 2019-2020.pdf


### Departments

In [49]:
departments_mappings = []
for file in tqdm(os.listdir('data/Ohio State')):
    mapping = get_departments(f"data/Ohio State/{file}")
    cleaned_mapping = dict()
    department_to_courses_ids_mapping = dict()
    
    for k,v in mapping.items():
        if len(v) > 50:
            cleaned_mapping[k] = v
            
    
    for k,v in cleaned_mapping.items():
        for r in re.findall(r'\n [0-9]{3,4}[\.0-9]{0,3}H?\n', v):
            department_to_courses_ids_mapping[r.strip()] = k
            
    try:
        departments_mappings.append(pd.Series(department_to_courses_ids_mapping, name=file))
    except:
        pass

<ipython-input-49-89da535d95b6>:17: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  departments_mappings.append(pd.Series(department_to_courses_ids_mapping, name=file))


KeyboardInterrupt: 

# UCLA

In [197]:

    
#     r"\n[A-Z]{0,2}[0-9]{2,4}[A-Z]{0,2}\.\ \ [.\.\s\w\n]*"
#     [\w\s\-\:\,\.\(\)\/.]*"
#     r"("
#     r"\n(?P<id>[A-Z]{0,2}[0-9]{2,4}[A-Z]{0,2})"                       # course ID
#     r"\.\ {1,2}"                                                      # spacing      
#     r"(?P<title>[A-Za-z:\s\n\.]*)"                                    # course title
#     r"\((?P<num>[0-9a-z\s\-]*)\)"                                       # magic number
#     r"(?P<decs>[.\n\s]*)"                                                            # course description
#     r")" #\.\ \(([0-9\ A-Z-a-z])\)"
# )


In [204]:
df_mapping[(df_mapping.college == 'UCLA') & (df_mapping.scrape_me == 'TRUE')]

,scrape_me,college,filename,first_department_name,first_department_page,last_department_page,example_course
0,TRUE,UCLA,UCLA_2017–18.pdf,AEROSPACE STUDIES – AIR FORCE ROTC,134,689,"CM240. Healing, Ritual, and Transformation. (4..."
1,TRUE,UCLA,UCLA_2009–10.pdf,AFRICAN STUDIES,122,629,599. Research for and Preparation of Ph.D. Dis...
2,TRUE,UCLA,UCLA_1997–99.pdf,AFRICAN AREA STUDIES,111,586,375. Teaching Apprentice Practicum (1 to 4 uni...
3,TRUE,UCLA,UCLA_2007–08.pdf,AFRICAN STUDIES,134,621,"451. Teaching Assistant Seminar. (2) Seminar, ..."
4,TRUE,UCLA,UCLA_1995–97.pdf,AFRICAN AREA STUDIES,104,555,100B. Psychology from an Afro-American Perspec...
5,TRUE,UCLA,UCLA_2013–14.pdf,AFRICAN STUDIES,122,668,"36. Composition, Rhetoric, and Language for ES..."
6,TRUE,UCLA,UCLA_2001–03.pdf,AFRICAN AREA STUDIES,131,584,"C474C. Dance of Spain. (2) Studio, three hours..."
7,TRUE,UCLA,UCLA_2008–09.pdf,AFRICAN STUDIES,122,621,M161. Comparative American Indian Societies. (...
8,TRUE,UCLA,UCLA_2010–11.pdf,AFRICAN STUDIES,124,627,C415. Advanced Modern/Postmodern Dance. (2) St...
9,TRUE,UCLA,UCLA_2016–17.pdf,AFRICAN AMERICAN STUDIES,131,688,M495J. Supervised Teaching of Writing for Engi...


In [206]:
from itertools import chain

# df_ucla = db2df(college=college)
final = []

for t in tqdm(df_mapping[(df_mapping.college == 'UCLA') & (df_mapping.scrape_me == 'TRUE')].itertuples()):
    filename = t.filename
    college = t.college, 
    dep = t.first_department_name
    first_page = t.first_department_page
    last_page = t.last_department_page
    df = df_ucla[df_ucla.filename == filename]
    data = df.pdfminer_detailed.iat[0]    
    print(filename, college, dep, first_page, last_page)
    for p in d[pg-1:]:
        if p['page'] == pg:
            i = ''.join(p['characters']['chars']).replace('\n','').index(dep)
            dep_font_size = p['characters']['size'][i:i+len(dep)]
            max_dep_font_size = max(dep_font_size)
            min_dep_font_size = min(dep_font_size)
            txt_font_size = max(p['characters']['size'][i-1:1+i+len(dep):len(dep)+1])
            break

    departments = dict()
    last_department = None

    for p in d[pg-1:]:
        for i,c in enumerate(p['characters']['size']):
            if c == max_dep_font_size:
                for j,cc in enumerate(p['characters']['size'][i:]):
                    if cc not in dep_font_size:
                        break
                if j != 0:
                    department = ''.join(p['characters']['chars'][i:i+j])
                    departments[department] = {'from': p['page'], 'to': None}
                    if last_department:
                        departments[last_department]['to'] = p['page'] - 1
                    last_department = department
                    j = 0
                    break

    departments[department]['to'] = last_page

    result = []

    regex_pattern_course_id = r"\n[A-Z]{0,2}[0-9]{2,4}[A-Z]{0,2}\.\ {1,2}"

    until_but_not_including = lambda rgx: rf"(?:(?!{rgx}).)*"
    everything_not_greedy = '.*?'
    regex_pattern_all_courses = (regex_pattern_course_id 
        + everything_not_greedy 
        + until_but_not_including(regex_pattern_course_id)) #, #+ "|" + until_but_not_including("\n\n"),)


    for department_name, pages in departments.items():
        department_text_block = ''.join(list(chain.from_iterable([
            p['tboxes']
            for p in d[ pages['from']-1 : pages['to']-1 ]
        ])))

        all_courses = re.findall(
            pattern=regex_pattern_all_courses,
            string=department_text_block, 
            flags=re.DOTALL
        )

        for course in all_courses:
            result.append((college, filename, department_name, course.replace('\n','')))

    df_result = pd.DataFrame(result, columns=['college','filename','department','course']).drop_duplicates()
    df_result['len'] = df_result.course.str.len()
    df_result = df_result[~df_result.department.str.contains("APPENDIX")]
    df_result.reset_index(inplace=True)
    df_result = df_result[~(df_result['len'] > 50000)] # TODO, e.g. df_result.at[255238,'course']
    final.append(df_result)


UCLA_2017–18.pdf ('UCLA',) AEROSPACE STUDIES – AIR FORCE ROTC 134 689



ValueError: substring not found

In [187]:

    lame = False
    if lame:
        with pd.ExcelWriter('data/ucla.xlsx') as writer:
            df_result.to_excel(writer, 'data')
    else:
        df2gsheet(df_result, college)
